# Feedback Prize EDA

NLPファンの皆さん、今頃またNLPのコンペがあるとは思いませんでしたが、いかがでしょうか？今回はどんなものか見てみましょう :) まずはデータを見て、それから主催者からの情報を見ていく、この順番が私にはベストです :) 

In [ ]:
import pandas as pd
import os
from pathlib import Path
import spacy
from spacy import displacy
from pylab import cm, matplotlib

### トレーニングデータ

テキストファイルの束と、ラベルを含む別の `train.csv` があるように見えます。まず、csvを見てみましょう。テキストファイルへの参照と、特定の **談話タイプ** を示すスパンを持つテキストファイルごとの複数行が得られます。以下では、談話のユニークなタイプも見てみましょう。

In [ ]:
train = pd.read_csv('../input/feedback-prize-2021/train.csv')
train.head()

In [ ]:
train.discourse_type.unique().tolist()

### `displacy` による可視化

csvファイルからテキストファイルにスパンを重ね合わせることができれば、最も効果的だと思います。Spacyには、このための素晴らしいビジュアライザーがあります。displacy`です。これを使って、1つの例がどのように見えるか見てみましょう!

In [ ]:
path = Path('../input/feedback-prize-2021/train')

colors = {
            'Lead': '#8000ff',
            'Position': '#2b7ff6',
            'Evidence': '#2adddd',
            'Claim': '#80ffb4',
            'Concluding Statement': 'd4dd80',
            'Counterclaim': '#ff8042',
            'Rebuttal': '#ff0000'
         }

def visualize(example):
    ents = []
    for i, row in train[train['id'] == example].iterrows():
        ents.append({
                        'start': int(row['discourse_start']), 
                         'end': int(row['discourse_end']), 
                         'label': row['discourse_type']
                    })

    with open(path/f'{example}.txt', 'r') as file: data = file.read()

    doc2 = {
        "text": data,
        "ents": ents,
        "title": example
    }

    options = {"ents": train.discourse_type.unique().tolist(), "colors": colors}
    displacy.render(doc2, style="ent", options=options, manual=True, jupyter=True)


In [ ]:
examples = train['id'].sample(n=5, random_state=42).values.tolist()

for ex in examples:
    visualize(ex)
    print('\n')

### インサイト

なるほど～、これは面白そうだ！」と思いました。
- いくつかの例では、テキスト全体が異なるカテゴリのスパンに密に分割されています。他の例では、アノテーターがいくつかの単語を省略し、分割が非常に主観的に見える。これは、アノテーションがノイズになっている可能性を示す指標です。
- リードで始まり、主張と証拠を混ぜ、結論の文で終わるという順序が重要なようです。これをモデルに取り入れる必要があるかもしれない。
- 同じクラスで2つのスパンが隣り合っている場合がある - 分離することが重要でしょう

### テキストの長さ

もう一つの重要なデータポイントは、テキストの長さです。彼らは`roberta`のようなモデルに適合するでしょうか？確認してみましょう

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [ ]:
texts = []

ids = train.sample(n=500)['id'].unique().tolist()
len(ids)

for example in ids:
    with open(path/f'{example}.txt', 'r') as file: data = file.read()
    texts.append({
        'text': data,
        'n_tokens': len(tokenizer(data)['input_ids'])
    })
     
df = pd.DataFrame(texts)

print(len(df[df.n_tokens > 512])/len(df))

df.n_tokens.hist();

**重要** 半数以上のテキストが標準的な最大長である512トークンを超えているため、スライディングウィンドウやその他のチャンキングアプローチをデータに適用する必要がありそうです。

### テキストのスパン数

また、1つのテキストに何個のスパンがあるか見てみましょう。また、スパンが最も少ないテキストと最も多いテキストを可視化してみましょう。

In [ ]:
dist = train.groupby('id')['discourse_type'].apply(lambda x: len(list(x)))

print(f'Min: {dist.min()}')
print(f'Max: {dist.max()}')

dist.hist();

In [ ]:
dist.sort_values()

In [ ]:
visualize('FFFF80B8CC2F')
visualize('FC7A3692794B')
visualize('149E8C278863')
visualize('71259B3EA87F')

### テスト

In [ ]:
sub = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')
sub.head()

なるほど、テストでも同じような形式のファイルを取得し、クラスと予測文字列を予測する必要があるようですね。あとは、ホストから提供される情報を見て、理解を深めるしかないですね :) 

> 単語のインデックスは、Pythonの.split()関数を使って計算し、結果のリストのインデックスを取ります。2つのオーバーラップは、グランドトゥルース/予測ペアのインデックスの各リストのset()を取り、各セットの長さで割った2つのセットの交点を計算することによって算出されます。

つまり、`predictionstring`フィールドに予測スパンに含まれる単語のインデックスを提供する必要があるということだと思います。

## ベースライン

これは面白いタスクだ! いくつかの方法でアプローチすることができます。
1. テキストを文単位で分割し、各文を分類する。
2. NER：個々のトークン（単語）を分類する。
3. 質問応答：談話の種類を質問として使用し、回答スパンを予測する。

それぞれのアプローチには、デメリットや複雑な点がありますが...。ベースラインを構築するのに十分なデータが揃ったと思います。別のノートブックでベースラインを作成し、後でこのEDAに戻ることにします。